<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4a69bd2cf90b3a4de9a0bcd3a1b73ea3fd58129bfeeebd8cce93756b91688b97
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-23 10:03:57
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.41 C
-------------------
Today PnL: -76.24 K (-0.54%)
Current PnL: -18.88 L (-12.7%)
CY Booked + Current PnL: -7.44 L (-5.01%)
-------------------
Total profit:  1.48 L
Total loss:  -20.36 L
-------------------
Total Booked + Current PnL: 19.54 L (16.05%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.13%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 84.75 L (60.23%)
Deployed:  1.22 C
Current:  1.41 C
CAGR/XIRR %: 8.9%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.77,61.0,H-MC,3.63,196500.0,31830.0,11495.0,-1.59,19.33,5.85,26.31,89.0,2.77,1.40,28.85,X40N,NTT,AC
77,TTKPRESTIG,770.00,97.23,41.0,M-SC,3.10,84986.0,-15791.0,15884.0,-1.23,-15.67,18.69,0.09,245.0,-0.99,0.60,11.65,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-44.49,59.0,H-MC,9.21,110330.0,9826.0,19286.0,-1.31,9.78,17.48,28.96,79.0,0.51,0.78,47.06,MH,ATH,METALS
57,RELIANCE,1533.00,-13.66,46.0,H-LC,4.66,216060.0,5694.0,23097.0,-0.40,2.71,10.69,13.68,37.0,0.25,1.54,19.68,XY25,NTT,REFINERIES
42,ITC,452.00,-38.77,41.0,H-LC,2.04,197532.0,-2606.0,23506.0,-0.74,-1.30,11.90,10.44,4.0,-0.11,1.40,4.19,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.0,3851.35,60.0,M-SC,5.36,123192.0,-24176.0,75997.0,3.99,-16.41,61.69,35.16,236.0,-0.32,0.88,25.83,XY24,NTT,AUTO
67,SONACOMS,1006.0,-39.99,42.0,M-SC,11.67,74708.0,-26451.0,101341.0,1.50,-26.15,135.65,74.03,202.0,-0.26,0.53,5.34,AR,BTT,AUTO
66,SIS,528.0,2071.50,35.0,H-SC,4.84,87644.0,-23388.0,46995.0,0.85,-21.06,53.62,21.26,156.0,-0.50,0.62,18.42,OX40N,NTT,MISC
73,TATAMOTORS,1065.0,-19.53,52.0,M-LC,2.30,254644.0,-21879.0,131957.0,0.75,-7.91,51.82,39.81,54.0,-0.17,1.81,22.04,XY24,NTT,AUTO
58,REPCOHOME,880.0,-59.76,50.0,H-SC,5.59,229803.0,-56702.0,314026.0,0.64,-19.79,136.65,89.82,134.0,-0.18,1.63,21.58,XY24,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1215.79,73.0,M-SC,8.02,179043.0,-2100.0,47572.0,-2.73,-1.16,26.57,25.10,235.0,-0.04,1.27,43.49,XY24,NTT,HEALTHCARE
45,KANSAINER,340.00,-67.79,53.0,H-SC,0.85,222309.0,-47358.0,83699.0,-2.37,-17.56,37.65,13.47,138.0,-0.57,1.58,12.44,XY24,NTT,PAINTS
5,ASIANPAINT,3465.66,-14.89,38.0,H-LC,7.64,214808.0,-36960.0,90176.0,-1.68,-14.68,41.98,21.13,27.0,-0.41,1.53,15.54,X40,ATH,PAINTS
8,AWL,485.00,-64.19,57.0,H-MC,3.67,254746.0,-47005.0,208433.0,-1.60,-15.58,81.82,53.50,116.0,-0.23,1.81,13.15,XY24,NTT,FMCG
14,BLUESTARCO,2080.00,13.77,61.0,H-MC,3.63,196500.0,31830.0,11495.0,-1.59,19.33,5.85,26.31,89.0,2.77,1.40,28.85,X40N,NTT,AC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.0,-20.42,48.0,H-MC,2.06,136922.0,1134.0,41624.0,-0.6,0.84,30.4,31.48,107.0,0.03,0.97,13.05,X40,ATH,INSURANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.0,97.23,41.0,M-SC,3.10,84986.0,-15791.0,15884.0,-1.23,-15.67,18.69,0.09,245.0,-0.99,0.60,11.65,OX40N,NTT,DURABLES
78,UJJIVANSFB,60.0,90.20,47.0,H-SC,9.25,120042.0,-22437.0,41955.0,-0.69,-15.75,34.95,13.70,163.0,-0.53,0.85,41.68,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.79,53.0,H-SC,0.85,222309.0,-47358.0,83699.0,-2.37,-17.56,37.65,13.47,138.0,-0.57,1.58,12.44,XY24,NTT,PAINTS
18,CERA,9475.0,-21.00,42.0,H-SC,2.07,144164.0,-31739.0,73754.0,-0.54,-18.04,51.16,23.89,149.0,-0.43,1.02,24.50,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-22.20,43.0,H-MC,7.09,105260.0,-26575.0,68798.0,-1.19,-20.16,65.36,32.03,98.0,-0.39,0.75,17.90,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,48.36,69.0,M-MC,8.92,234381.0,9419.0,158231.0,0.46,4.19,67.51,74.53,192.0,0.06,1.67,37.79,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.77,61.0,H-MC,3.63,196500.0,31830.0,11495.0,-1.59,19.33,5.85,26.31,89.0,2.77,1.40,28.85,X40N,NTT,AC
17,CAMS,5211.76,-1.62,58.0,H-SC,2.62,111944.0,9940.0,33986.0,-0.40,9.74,30.36,43.06,122.0,0.29,0.80,28.53,X40N,ATH,MISC
84,VOLTAS,1530.00,-1.92,46.0,H-MC,2.84,205665.0,13923.0,23837.0,-0.37,7.26,11.59,19.69,99.0,0.58,1.46,14.55,XY25,NTT,AC
23,DABUR,735.00,-7.26,40.0,H-MC,6.91,250866.0,2798.0,105615.0,-0.92,1.13,42.10,43.70,102.0,0.03,1.78,15.74,XY24,BTT,FMCG


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-44.49,59.0,H-MC,9.21,110330.0,9826.0,19286.0,-1.31,9.78,17.48,28.96,79.0,0.51,0.78,47.06,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-41.04,49.0,M-SC,1.14,99615.0,8116.0,70637.0,-0.08,8.87,70.91,86.07,223.0,0.11,0.71,44.66,XR,NTT,DURABLES
11,BANKINDIA,190.00,-29.19,63.0,H-MC,6.94,184114.0,4306.0,108112.0,-0.68,2.40,58.72,62.52,88.0,0.04,1.31,37.30,XR,NTT,BANKS
37,INDIAMART,4850.92,-53.89,31.0,H-SC,11.03,128938.0,5602.0,128164.0,-0.67,4.54,99.40,108.45,119.0,0.04,0.92,27.91,AR,ATH,MISC
86,WIPRO,420.00,-11.98,47.0,M-LC,5.68,154529.0,3584.0,105868.0,-0.44,2.37,68.51,72.51,53.0,0.03,1.10,8.54,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-18.08,37.0,H-LC,7.15,294852.0,-20990.0,133656.0,-0.39,-6.65,45.33,35.67,5.0,-0.16,2.10,6.29,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-14.89,38.0,H-LC,7.64,214808.0,-36960.0,90176.0,-1.68,-14.68,41.98,21.13,27.0,-0.41,1.53,15.54,X40,ATH,PAINTS
30,HCLTECH,1943.91,-1.51,39.0,H-LC,9.92,224447.0,-17477.0,80756.0,-0.72,-7.22,35.98,26.15,8.0,-0.22,1.60,6.06,X40,ATH,IT
19,CIPLA,1795.00,-21.49,41.0,H-LC,5.31,209130.0,4630.0,36786.0,-1.02,2.26,17.59,20.25,10.0,0.13,1.49,11.20,X40N,BTT,PHARMA
42,ITC,452.00,-38.77,41.0,H-LC,2.04,197532.0,-2606.0,23506.0,-0.74,-1.30,11.90,10.44,4.0,-0.11,1.40,4.19,X40,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-4.62,49.0,H-LC,1.48,236700.0,-9451.0,88668.0,0.00,-3.84,37.46,32.18,16.0,-0.11,1.68,30.82,X200,ATH,IT
42,ITC,452.0,-38.77,41.0,H-LC,2.04,197532.0,-2606.0,23506.0,-0.74,-1.30,11.90,10.44,4.0,-0.11,1.40,4.19,X40,NTT,FMCG
65,SIEMENS,4671.5,0.81,54.0,H-LC,3.33,162100.0,-23995.0,71470.0,-0.05,-12.89,44.09,25.51,15.0,-0.34,1.15,19.46,AR,ATH,ELECTRICAL
51,NESTLEIND,1377.0,-10.90,45.0,H-LC,4.11,274248.0,8822.0,47966.0,-1.37,3.32,17.49,21.40,11.0,0.18,1.95,10.19,XY25,NTT,FMCG
57,RELIANCE,1533.0,-13.66,46.0,H-LC,4.66,216060.0,5694.0,23097.0,-0.40,2.71,10.69,13.68,37.0,0.25,1.54,19.68,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-27.72,43.0,H-LC,12.16,287565.0,-58391.0,133258.0,-0.47,-16.88,46.34,21.64,1.0,-0.44,2.04,1.87,X40,ATH,IT
42,ITC,452.00,-38.77,41.0,H-LC,2.04,197532.0,-2606.0,23506.0,-0.74,-1.30,11.90,10.44,4.0,-0.11,1.40,4.19,X40,NTT,FMCG
80,UNITDSPR,1644.00,-12.80,52.0,H-LC,7.54,228622.0,-5522.0,54138.0,-0.80,-2.36,23.68,20.77,86.0,-0.10,1.63,4.40,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-1.51,39.0,H-LC,9.92,224447.0,-17477.0,80756.0,-0.72,-7.22,35.98,26.15,8.0,-0.22,1.60,6.06,X40,ATH,IT
83,VBL,671.64,-18.08,37.0,H-LC,7.15,294852.0,-20990.0,133656.0,-0.39,-6.65,45.33,35.67,5.0,-0.16,2.10,6.29,X40N,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.81,54.0,H-LC,3.33,162100.0,-23995.0,71470.0,-0.05,-12.89,44.09,25.51,15.0,-0.34,1.15,19.46,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,41.0,H-LC,2.04,197532.0,-2606.0,23506.0,-0.74,-1.30,11.90,10.44,4.0,-0.11,1.40,4.19,X40,NTT,FMCG
19,CIPLA,1795.00,-21.49,41.0,H-LC,5.31,209130.0,4630.0,36786.0,-1.02,2.26,17.59,20.25,10.0,0.13,1.49,11.20,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-14.89,38.0,H-LC,7.64,214808.0,-36960.0,90176.0,-1.68,-14.68,41.98,21.13,27.0,-0.41,1.53,15.54,X40,ATH,PAINTS
57,RELIANCE,1533.00,-13.66,46.0,H-LC,4.66,216060.0,5694.0,23097.0,-0.40,2.71,10.69,13.68,37.0,0.25,1.54,19.68,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-27.72,43.0,H-LC,12.16,287565.0,-58391.0,133258.0,-0.47,-16.88,46.34,21.64,1.0,-0.44,2.04,1.87,X40,ATH,IT
40,INFY,2275.00,-18.16,49.0,H-LC,8.12,319031.0,6143.0,165545.0,-0.11,1.96,51.89,54.87,3.0,0.04,2.27,8.74,X40,BTT,IT
42,ITC,452.00,-38.77,41.0,H-LC,2.04,197532.0,-2606.0,23506.0,-0.74,-1.30,11.90,10.44,4.0,-0.11,1.40,4.19,X40,NTT,FMCG
83,VBL,671.64,-18.08,37.0,H-LC,7.15,294852.0,-20990.0,133656.0,-0.39,-6.65,45.33,35.67,5.0,-0.16,2.10,6.29,X40N,ATH,FMCG
1,ABB,7934.00,-38.89,55.0,H-LC,9.02,255461.0,-6158.0,125380.0,-0.76,-2.35,49.08,45.57,7.0,-0.05,1.82,8.70,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6983.33,62.0,L-SC,5.28,77738.0,-16072.0,92695.0,-1.14,-17.13,119.24,81.67,269.0,-0.17,0.55,50.54,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-43.66,48.0,L-SC,42.26,82406.0,-31143.0,71191.0,-0.09,-27.43,86.39,35.27,268.0,-0.44,0.59,101.69,XR,NTT,HOTELS
61,SAMMAANCAP,326.00,-172.73,62.0,M-SC,4.49,83946.0,-18264.0,111657.0,-0.89,-17.87,133.01,91.37,208.0,-0.16,0.60,36.27,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-6.86,41.0,H-SC,12.04,86692.0,-14271.0,108798.0,-0.59,-14.14,125.50,93.63,148.0,-0.13,0.62,29.13,SR,ATH,CHEMICALS
49,MASFIN,398.61,-16.89,52.0,H-SC,12.68,94485.0,-3495.0,25095.0,0.03,-3.57,26.56,22.05,152.0,-0.14,0.67,37.34,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1215.79,73.0,M-SC,8.02,179043.0,-2100.0,47572.0,-2.73,-1.16,26.57,25.10,235.0,-0.04,1.27,43.49,XY24,NTT,HEALTHCARE
7,ATULAUTO,844.00,3851.35,60.0,M-SC,5.36,123192.0,-24176.0,75997.0,3.99,-16.41,61.69,35.16,236.0,-0.32,0.88,25.83,XY24,NTT,AUTO
61,SAMMAANCAP,326.00,-172.73,62.0,M-SC,4.49,83946.0,-18264.0,111657.0,-0.89,-17.87,133.01,91.37,208.0,-0.16,0.60,36.27,XY25,NTT,FINANCE
71,TANLA,1963.11,-16.63,68.0,H-SC,14.32,223105.0,-47656.0,377605.0,-0.82,-17.60,169.25,121.86,133.0,-0.13,1.59,74.53,AR,ATH,IT
81,VAIBHAVGBL,521.00,61.43,62.0,H-SC,6.02,139996.0,-42779.0,153842.0,-0.99,-23.41,109.89,60.77,125.0,-0.28,1.00,25.39,XR,NTT,JEWELLERY


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.31
1,25,43.68
2,50,74.77


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.10
LC    32.39
MC    23.57
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.39
X40      14.71
X40N     11.45
XR       11.27
XY25     10.05
AR        8.42
OX40N     7.81
X200      1.68
MH        1.64
X5K       1.41
SR        1.23
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.41
H-LC    25.93
H-MC    20.62
M-SC    13.18
M-LC     5.43
M-MC     2.65
L-SC     1.51
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.96,-4.48,38.97
IT,12.94,-15.54,78.53
FINANCE,10.08,-15.70,66.18
MISC,7.09,-15.41,79.57
BANKS,6.06,-17.11,78.72
PAINTS,5.61,-17.09,34.54
ELECTRICAL,5.50,-6.46,43.81
INSURANCE,3.70,-6.34,46.09
AC,3.56,3.18,13.03


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3107894.0,22
XR,1326981.0,14
AR,1149523.0,9
X40,779270.0,10
X40N,597650.0,9
OX40N,547102.0,10
XY25,425630.0,7
SR,249547.0,2
X5K,119639.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3658754.0,29
M-SC,1394285.0,17
H-MC,1205350.0,15
H-LC,1175237.0,15
M-LC,386014.0,4
M-MC,305830.0,2
L-SC,249717.0,3
L-MC,60996.0,1
L-LC,38839.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1227498.0      6
M-SC       XY24       797283.0      7
H-SC       AR         788549.0      5
           XR         772771.0      7
H-MC       XY24       552185.0      4
H-LC       X40        493241.0      5
M-MC       XY24       305830.0      2
H-SC       X40N       297564.0      4
           OX40N      258993.0      4
           SR         249547.0      2
H-LC       X40N       224580.0      3
H-MC       X40        213636.0      4
H-LC       AR         196850.0      2
H-MC       XY25       167827.0      2
M-SC       AR         164124.0      2
L-SC       XR         163886.0      2
M-LC       XY24       160507.0      2
M-SC       OX40N      133480.0      4
M-LC       X5K        119639.0      1
M-SC       XR         115348.0      2
           XY25       111657.0      1
H-MC       XR         108112.0      1
H-LC       XY25       107307.0      3
M-LC       XR         105868.0      1
H-LC       X200        88668.0      1
L-SC       OX40N       85831.0      1
H-MC       X40N        75506.0      2
M-SC       X40         72393.0      1
H-MC       OX40N       68798.0      1
H-LC       XY24        64591.0      1
H-SC       MH          63832.0      1
L-MC       XR          60996.0      1
L-LC       XY25        38839.0      1
H-MC       MH          19286.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 49.0 seconds
